# 1. Introduction

Classic Neural Network lack the possibility to know how confident the model is when making a prediction. That can be problematic in critical fields. 

If we want to quantify the belief about a prediction, we need the distribution $p(w/D)$ which represents how likely a set of parameters is for a given dataset. By having this distribution, we can now get the prediction confidence distribution $p(y/x, D)$ which is the expectancy of the prediction confidence for each model which parameters w follow $p(w/D)$. 

So, $$p(y/x, D) = E_{p(w/D)}[p(y/x, w)]$$

In pratice, if we want to make a prediction, we just sample N time a set of parameters w and we average the prediction.

# 2. Intractable distribution

The problem we encounter is that $p(w/D) = \frac{p(D/w)p(w)}{p(D)}$ is too hard to compute because of the evidence $p(D) = \int p(D/w)p(w)dw$ : There are too many parametrization possible. We can either use MCMC algorithm or variational inference to bypass the issue. We'll take the latter.

# 3. Variational Inference

### 3.1 Kullback divergence

The idea is to approximate $p(w/D)$ with a variational posterior distribution $q(w/\theta)$. $\theta$ represents the variational parameters of that distributions.


To measure how close the two distributions are, we'll use kullback divergence $$D_{KL}(P//Q) = \int P(x)log(\frac{P(x)}{Q(x)})dx$$

Kullback divergence comes from information theory, here is some context about it :

$I(x) = -log(P(x))$ represents the information of the event x with distribution P, the less likely the event x is to occur, the more information it carries.

$H(x) = E_{P(x)}[-log(P(x))]$ represents the average information you'll get out of a set of event x for the distribution P.

So in the end, $D_{KL}(P//Q)$ measures how much extra "coding cost" (in bits) you would have if you used codes optimized for a distribution (Q) instead of the approximate distribution (P) and that is why we use it as a measure.



### 3.2 Loss function

So we want to minimize $D_{KL}(q(w/\theta)//p(w/D))$ which means we want to find $\theta$ so that it verifies :

\begin{equation*}
\begin{split}
\theta^{*} & = \arg\min_{\theta} \text{KL}[q(\mathbf{w}\ |\ \mathbf{\theta})\ ||\ P(\mathbf{w}\ |\ \mathcal{D}]\\
& = \arg\min_{\theta} \int q(\mathbf{w}\ |\ \mathbf{\theta}) \log \frac{q(\mathbf{w}\ |\ \mathbf{\theta})}{P(\mathbf{w}) P(\mathcal{D}\ |\ \mathbf{w})} d\mathbf{w} \\
& = \arg\min_{\theta} \text{KL}[q(\mathbf{w}\ |\ \mathbf{\theta})\ ||\ P(\mathbf{w})] - \mathbb{E}_{q(\mathbf{w}\ |\ \mathbf{\theta})}[\log P(\mathcal{D}\ |\ \mathbf{w})]
\end{split}
\end{equation*}

The left term represents how far we get from the prior so it penalizes distributions that become too complex and the right term represents in average how good we fit the data with the parameters following $q(w/\theta)$.

As written in the paper, we will define the loss function as :

\begin{equation*}
\mathcal{F}(\mathcal{D}, \mathbf{\theta}) = \text{KL}[q(\mathbf{w}\ |\ \mathbf{\theta})\ ||\ P(\mathbf{w})] - \mathbb{E}_{q(\mathbf{w}\ |\ \mathbf{\theta})}[\log P(\mathcal{D}\ |\ \mathbf{w})]
\end{equation*}

To speed up greatly the computation, we will approximate the divergence and the expectancy :

\begin{equation*}
\mathcal{F}(\mathcal{D}, \mathbf{\theta}) \approx \sum_{i = 1}^{n} \log q(\mathbf{w}^{(i)}\ |\ \mathbf{\theta}) - \log P(\mathbf{w}^{(i)}) - \log P(\mathcal{D}\ |\ \mathbf{w}^{(i)})
\end{equation*}

In practice, we will only sample one weight to compute the loss function. Because of that, the graph of the loss function regarding the epoch will be noisy.

# 4. Modeling the variational posterior, prior and likelihood disitrubtions

Now that we have the loss function, we need to model the variational posterior $q(w/\theta)$, prior $P(w)$ and the likelihood $P(D/w)$. 

As usual, the Gaussian distribution will be used because of its universality. 

A little tweak for the prior is that we use two Gaussian and not only one so the distribution will be a Scale Gaussian Mixture. It rewards the weights to be close to 0 while not penalizing too much for not be close to 0 which yields better result in practice. So we get : 

\begin{equation*}
P(\mathbf{w}) = \prod_i \bigg ( \pi \mathcal{N}(\mathbf{w}_i\ |\ 0,\sigma_1^2) + (1 - \pi) \mathcal{N}(\mathbf{w}_i\ |\ 0,\sigma_2^2)\bigg )
\end{equation*}

where $\pi \in [0, 1]$, $\sigma_1 > \sigma_2$ and $\sigma_2 \ll 1$.

\begin{equation*}
P(\mathbf{w}\ |\ D) = \prod_i \mathcal{N}(\mathbf{y}_i\ |\,\mathbf{NN(xi)},\mathbf{\sigma_{likelihood}}^2)
\end{equation*}

where $NN$ is the functional model.

\begin{equation*}
q(\mathbf{w}\ |\ \theta) = \prod_i \mathcal{N}(\mathbf{w}_i\ |\ \mathbf{\mu},\mathbf{\sigma}^2)
\end{equation*}

The parameters $\theta$ we want to find are those used for the variational posterior distribution, the other distributions have fixed parameters that you choose beforehand.

# 5. Conclusion and remarks

So now we have everything in hand to make the Bayesian Neural Network. We will now have to train the network by computing gradients and optimize the parameters. This notebook is not at all comprehensive of all the theory elements to know, check the referenced papers if you want more deep understanding about the theory.

Some remarks : I noticed when I experimented the BNN that the training is very volatile and the gradient can diverge really quickly because the scales are very small ($10^{-2}$ to $10^{-4}$) so I added the classic MSE loss in the training, which is less "explosive" and help to converge. Also, the learning rate have to be very small because of that "explosiveness" (0.001 to 0.00001).